# Módulo 3: Ejercicio de evaluación final - ELISABET AGUILÓ THIÓ

## Los Datos

Los datos que se proporcionan consisten en dos datasets que, en conjunto, describen el comportamiento de los clientes dentro de un programa de lealtad de una aerolínea.

---

## 1. Customer Flight Analysis.csv

**Información sobre la actividad de vuelo de los clientes.**

| Atributo | Descripción |
|----------|-------------|
| **Loyalty Number** | Identificador único del cliente dentro del programa de lealtad. |
| **Year** | Año en el cual se registraron las actividades de vuelo. |
| **Month** | Mes del año (1 a 12) en el cual ocurrieron las actividades de vuelo. |
| **Flights Booked** | Número total de vuelos reservados en ese mes. |
| **Flights with Companions** | Número de vuelos reservados con acompañantes. |
| **Total Flights** | Total de vuelos realizados por el cliente (incluye meses anteriores). |
| **Distance** | Distancia total volada durante el mes (millas o kilómetros). |
| **Points Accumulated** | Puntos acumulados durante el mes. |
| **Points Redeemed** | Puntos redimidos durante el mes. |
| **Dollar Cost Points Redeemed** | Valor en dólares de los puntos redimidos durante el mes. |

---

## 2. Customer Loyalty History.csv

**Perfil detallado de los clientes.**

| Atributo | Descripción |
|----------|-------------|
| **Loyalty Number** | Identificador único del cliente dentro del programa de lealtad. **Permite correlacionar** con el archivo de actividad de vuelos. |
| **Country** | País de residencia del cliente. |
| **Province** | Provincia o estado de residencia (aplicable a países con divisiones provinciales o estatales). |
| **City** | Ciudad de residencia del cliente. |
| **Postal Code** | Código postal del cliente. |
| **Gender** | Género del cliente (ej. Male o Female). |
| **Education** | Nivel educativo alcanzado (ej. Bachelor, College, etc.). |
| **Salary** | Ingreso anual estimado del cliente. |
| **Marital Status** | Estado civil (ej. Single, Married, Divorced, etc.). |
| **Loyalty Card** | Tipo de tarjeta de lealtad (niveles o categorías dentro del programa). |
| **CLV (Customer Lifetime Value)** | Valor estimado que el cliente aporta durante toda la relación con la empresa. |
| **Enrollment Type** | Tipo de inscripción en el programa (ej. Standard). |
| **Enrollment Year** | Año de inscripción en el programa de lealtad. |
| **Enrollment Month** | Mes de inscripción en el programa de lealtad. |
| **Cancellation Year** | Año de cancelación de la membresía, si aplica. |
| **Cancellation Month** | Mes de cancelación de la membresía, si aplica. |



## Fase 1: Exploración y Limpieza

## Librerías utilizadas

In [ ]:
# ------------------------------
# Manipulación de datos
# ------------------------------
import pandas as pd        # pandas → trabajar con tablas (DataFrames)
import numpy as np         # numpy → operaciones numéricas y manejo de nulos (np.nan)
import os                  # os → interacción con el sistema operativo (verificar existencia de archivos)

# ------------------------------
# Configuración del entorno
# ------------------------------
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en la salida
pd.set_option('display.max_rows', 100)      # Limitar filas mostradas a 100 (para evitar salidas excesivas)


## Metodología de Exploración y Limpieza Inicial
**Siempre exploramos y limpiamos cada CSV *individualmente* antes de combinarlos.**

Cada archivo CSV puede tener:
- Columnas diferentes
- Datos que no coinciden 
- Espacios vacíos

**Explorar y limpiar por separado evita errores y deja datos correctos para análisis.**
